### Modules and data

In [1]:
# Libraries
import matplotlib.pyplot as plt
import numpy as np
import gurobipy as gb
import os
from pickle import dump, load
from time import process_time
from copy import deepcopy
from source import *
from multiprocess import pool

path = os.getcwd()

In [2]:
# Load data
file = open(path+"/Results/Configurations/Open Stations/open_stations_18","rb"); S = load(file); file.close()
K_s = {}                # {scenario:{station:[vehicles]}}n",
K = {}                  # {scenario:[vehicles]}n",

for sc in range(25):
    file = open(path+f"/Data/K/K_sc{sc}", "rb")
    K[sc] = load(file); file.close()

    file = open(path+f"/Data/K_s10/Ks_sc{sc}", "rb")
    K_s[sc] = load(file); file.close()

original_stress = {}
for s in S:
    original_stress[s] = np.mean(np.array([len(K_s[sc][s]) for sc in range(25)]))

### Auxiliary functions

In [3]:
# Auxiliary functions - Number of chargers
def generate_routes_given_number_chargers_individual(s,S,y,K,K_s_prime,max_chargers=8,exhaust=False):

    total_serviced_costumers:dict[list[list]] = dict()
    
    for sc in range(25):
        routes = list()
        K, _, S_k, a, t = load_pickle(path+"/Data/",sc)
        K_s = K_s_prime[sc]

        S_k = {k:[ss for ss in S_k[k] if ss in S] for k in K_s[s]}
        a.update({("s",s):0}); a.update({("e",s):40})
        t.update({("s",s):0}); t.update({("e",s):0})
    
        mp = gb.Model("Restricted Master Problem")
    
        dummy_0 = {k:mp.addVar(vtype=gb.GRB.CONTINUOUS, obj=1, name=f"st0_{k}") for k in K_s[s]}
        aux = mp.addVar(vtype=gb.GRB.CONTINUOUS, obj=2, name=f"aux_{s}")
    
        vehic_assign = {}
        for k in K_s[s]:
            vehic_assign[k] = mp.addLConstr(dummy_0[k] == 1, name=f"V{k}_assignment")
    
        st_conv = mp.addConstr(aux <= y, f"S{s}_convexity")
    
        mp.setParam("OutputFlag",0)
    
        time0 = process_time(); i = 0
        lbd = []; lastMP = 1e3
    
        while True:
            
            mp.update()
            mp.optimize()
    
            pi = {k:vehic_assign[k].getAttr("Pi") for k in K_s[s]}
            sigma = st_conv.getAttr("Pi")
            thisMP = mp.getObjective().getValue()
    
            if (lastMP-thisMP < 0.1 and thisMP > 5) or (lastMP-thisMP < 0.01 and thisMP<=5):
                #if y == 8: print(f"ttOut of Labeling scenario {sc}")
                mpsol = thisMP; break
    
            #print(f"tIteration {i}:ttMP obj: {round(mp.getObjective().getValue(),2)}ttime: {round(process_time()-time0,2)}s")
            i += 1
    
            V,A,rc = get_graph_chargers(s,K_s[s],a,pi,sigma)
            ext = vertices_extensions(V,A)
            
            #if y == 8:
            #    print(f"ttIteration {i}. Labeling for scenario {sc}")
    
            routes_DFS = {"s":([0],0)}; covered_nodes = {v:0 for v in V}; best_len = {v:0 for v in V}; best_t = {v:0 for v in V}; best_h = {v:30 for v in V}; num_improv = {v:0 for v in V}; tried = {v:0 for v in V}
            marked = {}; max_successors = {v:100 for v in V}
            mod_label_DFS(v="s",rP=0,tP=0,qP=0,hP=0,P=[],cK=covered_nodes,L=routes_DFS,s=s,r=rc,t=t,a=a,ext=ext,best_len=best_len,best_t=best_t,best_h=best_h,tried=tried,marked=marked,num_improv=num_improv,max_successors=max_successors)
            del routes_DFS["s"]
            
            opt = 0
            for l in routes_DFS:
                if routes_DFS[l][1] < -0.001:
                    col = {k:1 if k in routes_DFS[l][0] else 0 for k in K_s[s]}
                    new_Col = gb.Column(col.values(),vehic_assign.values())
                    lbd.append(mp.addVar(vtype=gb.GRB.CONTINUOUS,obj=0,column=new_Col))
    
                    mp.chgCoeff(st_conv,lbd[-1],1)
                    opt +=1
            if i == 1:
                original_routes = routes_DFS
    
            if opt == 0 : mpsol = thisMP; break
            lastMP = thisMP
        
        for v in mp.getVars():
            v.vtype = gb.GRB.BINARY
        #if mpsol < len(K_s[s]): mp.setParam("MIPGap",0.01)
        mp.setParam("MIPFocus",2)
        mp.update(); mp.optimize()
    
        # print(f"ttSolved IMP scenario {sc}, with {mp.getObjective().getValue()} unnasigned vehicles")
        # print(f"IMP solution: {mp.getObjective().getValue()} assigned vehicles with {y} chargers")
        for l in lbd:
            if l.X > 0.5:
                col = [k for k in K_s[s] if mp.getCoeff(vehic_assign[k],l) == 1]
                routes.append(col)
    
        total_serviced_costumers[sc]:list = list()
    
        for route in routes:
            total_serviced_costumers[sc].extend(route)
        
        if len(total_serviced_costumers[sc]) < len(K_s_prime[sc][s]) and y < max_chargers:
            # print(f't{y} number of chargers is infeasible')
            return None, None
    
    return routes, total_serviced_costumers


def generate_routes_given_number_chargers(S,y,K_prime,K_s_prime,exhaust=False):

    total_serviced_costumers:dict[dict[list]] = dict()

    sigmass = dict()
    sigmass.update({s:0 for s in S})

    missing = False
    
    for sc in range(25):
        routes = list()
        _, _, S_k, a, t = load_pickle(path+"/Data/",sc)
        for s in S:
            a.update({("s",s):0}); a.update({("e",s):40})
            t.update({("s",s):0}); t.update({("e",s):0})

        K = K_prime[sc]
        K_s = K_s_prime[sc]
    
        routes, sigmas = second_stage_chargers(S, K, K_s, y, a, t, sc)
        sigmass[s] += sigmas[s]/25

        total_serviced_costumers[sc]:dict = dict()
        total_total:list = list()
        for s in S:
            total_serviced_costumers[sc][s]:list = list()
            for route in routes[s]:
                total_serviced_costumers[sc][s].extend(route)
                total_total.extend(route)

        # for s in S:
        #     for k in K_s_prime[sc][s]:
        #         if k not in total_total and y[s] < 8:
        #             missing = True
        #             sigmass = sigmas
        #             break
        #     if missing:
        #         break
            
        # if missing:
        #     break
        
    return routes, total_serviced_costumers, sigmass, missing


def generate_initial_routes(S, y, K_prime, K_s_prime, exhaust=False):

    total_serviced_costumers:dict[dict[list]] = dict()

    sigmass = dict()
    sigmass.update({s:0 for s in S})

    def generate_routes_for_scenario(sc):
        routes = list()
        _, _, S_k, a, t = load_pickle(path+"/Data/",sc)
        for s in S:
            a.update({("s",s):0}); a.update({("e",s):40})
            t.update({("s",s):0}); t.update({("e",s):0})

        K = K_prime[sc]
        K_s = K_s_prime[sc]
    
        routes, sigmas = second_stage_chargers(S, K, K_s, y, a, t, sc)
        sigmass[s] += sigmas[s]/25

        total_serviced_costumers[sc]:dict = dict()
        total_total:list = list()
        for s in S:
            total_serviced_costumers[sc][s]:list = list()
            for route in routes[s]:
                total_serviced_costumers[sc][s].extend(route)
                total_total.extend(route)
        
        print(f'Initial {sc} scenario done')
        return (sc, routes, total_serviced_costumers, total_total)

    lab = pool.Pool(                                             )
    Results = lab.map(generate_routes_for_scenario, range(25))

    return Results

In [4]:
# Auxiliary functions - Clusters
def compute_shared_vehicles(S, K_s): 
    shared_vehicles = dict()
    for s in S:
        for ss in S:
            if s != ss:
                shared_vehicles[s,ss] = sum(len(set(K_s[sc][s]).intersection(set(K_s[sc][ss]))) for sc in range(25))/25
                # shared_vehicles[s,ss] = sum([1 for sc in range(25) for k in K_s[sc][s] if k in K_s[sc][ss]])/25
    return shared_vehicles


def get_associated_stations(S, K_s, station):
    # Get average number of shared vehicle per station
    shared_vehicles = compute_shared_vehicles(S, K_s)

    # Filter the dictionary to get the key-value pairs with matching i
    filtered_dict = {key: value for key, value in shared_vehicles.items() if key[0] == station}
    
    # Sort the filtered dictionary by values in descending order
    sorted_dict = sorted(filtered_dict.items(), key=lambda x: x[1], reverse=True)
    
    # Extract the top four j values from the sorted dictionary
    top_j_values = [key[1] for key, value in sorted_dict[:4]]

    # Locate vehicles assigned to the stations
    K = dict()
    for sc in range(25):
        K[sc] = list()
        for s in [station]+top_j_values:
            for k in K_s[sc][s]:
                if k not in K[sc]:
                    K[sc].append(k)

    return shared_vehicles, top_j_values, K

# Give chargers to stations given their stress index (individually)

In [5]:
start = process_time()

ordered_stations:list = sorted(original_stress, key = original_stress.get, reverse = True)     # Get list of stations ordered by stress
stress = deepcopy(original_stress)

y:dict = dict(); y.update({s:0 for s in S})
Results = dict()
Results.update({sc:{'routes': dict(), 'total_serviced_costumers': dict(), 'total_total':list()} for sc in range(25)})

K_prime = deepcopy(K)
K_s_prime = deepcopy(K_s) 

cont:int = 0

while True:
    cont += 1 
    s = ordered_stations[0]
    y[s] = 1       # Start giving one charger to the station

    # Iterative process to find number of chargers 
    while True:
        routes, total_serviced_costumers = generate_routes_given_number_chargers_individual(s, S, y[s], K_prime, K_s_prime)     # Get the routes for the stations
        
        if routes == None:  # Station doesn't service all possible costumers in at least one scenario
            y[s] += 1
        else:               # Number of chargers is fixed
            break

    if cont == 1: print(f'Iter \tSt \t#ch \tStress \tProgr \tTime \n------------------------------------------------------')
    print(f'{cont} \t{s} \t{y[s]} \t{round(stress[s],2)} \t{round(round(cont/600,2)*100,4)}% \t{round(process_time()-start,2)}')    

    # Save routes and remove serviced vehicles
    for sc in range(25):
        Results[sc]['routes'][s] = routes
        Results[sc]['total_serviced_costumers'][s] = total_serviced_costumers[sc]

        for vehicle in total_serviced_costumers[sc]:
            Results[sc]['total_total'].append(vehicle)
            K_prime[sc].remove(vehicle)
        
            for ss in S:
                if vehicle in K_s_prime[sc][ss]:
                    K_s_prime[sc][ss].remove(vehicle)

    S.remove(s)

    # Recompute stress index
    stress:dict = {}
    for s in S:
        stress[s] = np.mean(np.array([len(K_s_prime[sc][s]) for sc in range(25)]))    
    ordered_stations:list = sorted(stress, key = stress.get, reverse = True)     # Get list of stations ordered by stress

    # Stoping condition: All stations are assigned or no vehicles are left on any scenario
    if cont == 600 or sum(len(K_prime[sc]) for sc in range(25)) == 0:
        break

Results['Chargers'] = y

a_file = open(path+'/Results/Number of chargers/Greedy assignment', "wb")
pickle.dump(Results, a_file)
a_file.close()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-11


/Users/juanbeta/My Drive/Research/MOPTA/MOPTA-23/source.py:212: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  cK.update({k:1 if k in li else 0 for k in cK})


Iter 	St 	#ch 	Stress 	Progr 	Time 
------------------------------------------------------
1 	379 	8 	196.12 	0.0% 	29.34
2 	889 	8 	193.0 	0.0% 	59.63
3 	1154 	8 	169.0 	1.0% 	86.78
4 	207 	8 	153.92 	1.0% 	108.89
5 	808 	8 	148.44 	1.0% 	131.65
6 	113 	8 	133.68 	1.0% 	154.22
7 	46 	8 	113.8 	1.0% 	166.88
8 	1033 	8 	113.64 	1.0% 	182.75
9 	170 	8 	99.88 	1.0% 	201.57
10 	1047 	8 	97.12 	2.0% 	211.85
11 	408 	8 	90.48 	2.0% 	224.16
12 	309 	8 	88.36 	2.0% 	234.66
13 	373 	8 	85.68 	2.0% 	247.34
14 	191 	8 	83.2 	2.0% 	255.8
15 	1135 	8 	82.12 	3.0% 	267.66
16 	340 	8 	79.24 	3.0% 	278.48
17 	126 	8 	76.32 	3.0% 	287.94
18 	1030 	8 	76.16 	3.0% 	296.47
19 	356 	8 	75.64 	3.0% 	310.01
20 	10 	8 	75.48 	3.0% 	318.71
21 	290 	8 	73.56 	4.0% 	330.06
22 	921 	8 	71.2 	4.0% 	338.31
23 	284 	8 	68.88 	4.0% 	347.8
24 	885 	8 	67.52 	4.0% 	355.68
25 	1077 	8 	65.92 	4.0% 	371.41
26 	1049 	8 	64.88 	4.0% 	378.75
27 	6 	8 	64.72 	4.0% 	386.33
28 	1216 	8 	64.44 	5.0% 	395.79
29 	267 	8 	60.76 	5

# Give chargers to stations given their dual variables (cluster)

In [ ]:
ordered_stations:list = sorted(original_stress, key = original_stress.get, reverse = True)     # Get list of stations ordered by stress
number_of_chargers:dict = dict()
stress = deepcopy(original_stress)

K_prime = deepcopy(K)
K_s_prime = deepcopy(K_s) 

cont:int = 0

while True:
    cont += 1
    s = ordered_stations[0]
    shared_vehicles, compatible_stations, K_prime = get_associated_stations(S, K_s_prime, s)
    print(f'Cluster # {cont}: {compatible_stations}') 
    
    y:dict = {st:1 for st in compatible_stations}       # Start giving one charger to the station

    # Iterative process to find number of chargers 
    while True:
        print(f"\tTrying for {list(y.values())} configuration")
        total_serviced_costumers, sigmas, missing = generate_routes_given_number_chargers(compatible_stations, y, K_prime, K_s_prime, False)     # Get the routes for the stations
        
        if missing:
            sigmas = {key:value for key,value in sigmas.items() if y[key]<8}
            best_station = min(sigmas, key = sigmas.get)
            y[best_station] += 1
        else:
            break
    
    # Set number of chargers and remove already serviced vehicles
    for st in compatible_stations:
        number_of_chargers[st] = y[st]
        print(f'\n\tStation {st} set to {number_of_chargers[st]} chargers\nn----------------------------')

        for u in range(25):
            for vehicle in total_serviced_costumers[u][st]:
                K_prime[u].remove(vehicle)
            
                for ss in S:
                    if vehicle in K_s_prime[u][ss]:
                        K_s_prime[u][ss].remove(vehicle)

        S.remove(st)

    stress:dict = {}
    for s in S:
        stress[s] = np.mean(np.array([len(K_s_prime[sc][s]) for sc in range(25)]))    
    ordered_stations:list = sorted(stress, key = stress.get, reverse = True)     # Get list of stations ordered by stress

    if cont == 600 or sum(len(K_prime[sc]) for sc in range(25))==0:
        break

# Two phase assignment: Give one charger to each station and then assing given their stress index

In [5]:
start = process_time()
initial_routes = 'load' #'generate'

# Duplicate parameters
K_prime = deepcopy(K)
K_s_prime = deepcopy(K_s) 
ordered_stations:list = sorted(original_stress, key = original_stress.get, reverse = True)     # Get list of stations ordered by stress

# Give one charger to each station
y:dict = dict(); y.update({s:1 for s in S})

# Generate initial routes (one charger per station)
if initial_routes == 'generate': 
    Results = generate_initial_routes(S, y, K_prime, K_s_prime, False)
    print(f'\n\t--------- Finished initial assignment at {round(process_time()-start,s)} --------- \n')
elif initial_routes == 'load': 
    file = open(path+'/Results/Number of chargers/Initial Routes', 'rb'); Results = pickle.load(file); file.close()
    print(f'\n\t--------- Loaded initial assignment --------- \n')

Results = {i[0]:{'routes':i[1], 'total_serviced_costumers':i[2][i[0]], 'total_total':i[3]} for i in Results}

# Fix serviced vehicles per each station
for sc, values in Results.items():
    for vehicle in values['total_total']:
        K_prime[sc].remove(vehicle)
        for s in S:
            if vehicle in K_s_prime[sc][s]:
                K_s_prime[sc][s].remove(vehicle)
                

# If a station no longer has availabe vehicles on any scenario, remove. Else, compute new stress index
stress:dict = dict()
for s in S:
    if sum(len(K_s_prime[sc][s]) for sc in range(25))==0:
        S.remove(s)
    else:
        stress[s] = np.mean(np.array([len(K_s_prime[sc][s]) for sc in range(25)]))    

ordered_stations:list = sorted(stress, key = stress.get, reverse = True)     # Get list of stations ordered by stress

# Iterative process
cont:int = 600-len(S)
second_cont = 0
while True:
    cont += 1
    second_cont += 1
    s = ordered_stations[0]

    y_aux = 1

    # Iterative process to find number of chargers 
    while True:
        routes, total_serviced_costumers = generate_routes_given_number_chargers_individual(s, S, y_aux, K_prime, K_s_prime, 7)     # Get the routes for the stations
        
        if routes == None:
            y_aux += 1
        else: 
            break
    
    if second_cont == 1: print(f'Iter \tSt \t#ch \tStress \tProgr \tTime \n------------------------------------------------------')
    y[s] += y_aux
    print(f'{second_cont} \t{s} \t{y[s]} \t{round(stress[s],2)} \t{round(round(cont/600,2)*100,4)}% \t{round(process_time()-start,2)}')    

    # Save routes and remove serviced vehicles
    for sc in range(25):
        Results[sc]['routes'][s].extend(routes)

        for vehicle in total_serviced_costumers[sc]:
            Results[sc]['total_serviced_costumers'][s].append(vehicle)
            Results[sc]['total_total'].append(vehicle)

            K_prime[sc].remove(vehicle)
        
            for ss in S:
                if vehicle in K_s_prime[sc][ss]:
                    K_s_prime[sc][ss].remove(vehicle)

    S.remove(s)

    # Recompute stress index
    stress:dict = dict()
    for s in S:
        stress[s] = np.mean(np.array([len(K_s_prime[sc][s]) for sc in range(25)]))    
    ordered_stations:list = sorted(stress, key = stress.get, reverse = True)     # Get list of stations ordered by stress

    # Stoping condition: All stations are assigned or no vehicles are left on any scenario
    if cont == 600 or sum(len(K_prime[sc]) for sc in range(25)) == 0:
        break

Results['Chargers'] = y

a_file = open(path+'/Results/Number of chargers/Two phase assignment', "wb")
pickle.dump(Results, a_file)
a_file.close()


	--------- Loaded initial assignment --------- 

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-11


/Users/juanbeta/My Drive/Research/MOPTA/MOPTA-23/source.py:212: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  cK.update({k:1 if k in li else 0 for k in cK})


Iter 	St 	#ch 	Stress 	Progr 	Time 
------------------------------------------------------
1 	379 	8 	125.72 	2.0% 	34.39
2 	889 	8 	84.32 	2.0% 	45.96
3 	1124 	8 	82.24 	2.0% 	56.03
4 	207 	8 	77.52 	3.0% 	66.73
5 	808 	8 	76.84 	3.0% 	75.81
6 	113 	8 	48.36 	3.0% 	85.85
7 	373 	8 	45.88 	3.0% 	91.43
8 	405 	8 	43.32 	3.0% 	97.69
9 	1033 	7 	43.32 	3.0% 	105.58
10 	46 	7 	40.44 	4.0% 	110.63
11 	217 	8 	38.44 	4.0% 	118.07
12 	951 	6 	36.76 	4.0% 	123.35
13 	279 	8 	35.64 	4.0% 	128.35
14 	19 	8 	31.92 	4.0% 	133.67
15 	408 	8 	30.48 	4.0% 	138.0
16 	290 	7 	28.44 	4.0% 	141.97
17 	424 	8 	27.84 	5.0% 	146.47
18 	1047 	7 	27.72 	5.0% 	151.94
19 	10 	7 	27.48 	5.0% 	159.63
20 	340 	6 	27.48 	5.0% 	165.98
21 	1135 	6 	26.64 	5.0% 	169.64
22 	13 	6 	25.56 	6.0% 	173.28
23 	885 	6 	23.52 	6.0% 	178.29
24 	318 	8 	23.12 	6.0% 	181.58
25 	193 	5 	20.68 	6.0% 	184.09
26 	1052 	5 	19.84 	6.0% 	186.61
27 	1146 	5 	19.8 	6.0% 	190.15
28 	446 	5 	19.72 	7.0% 	193.69
29 	1216 	5 	19.48 	7.0% 	196

# Retrieving information

In [5]:
# Auxiliary functions - Upload and display results

# Retrieve results
def upload_data(heuristic):
    file = open(path + f'/Results/Number of chargers/{heuristic}', 'rb')
    Results = pickle.load(file)
    file.close()

    return Results

# Print stats of solution
def print_general_information(heuristic, Results):
    print(f'---------------- Heuristic {heuristic} ----------------')
    print(f'- Number of active stations: {sum([1 for value in Results["Chargers"].values() if value != 0])}')
    print(f'- Number of chargers: {sum(Results["Chargers"].values())} \n')

# Export Results
def export_num_chargers(Results_greedy, Results_two_phase):
    data = dict(); data.update({'station':S})

    data['greedy_chargers'] = list();   data['two_phase_chargers'] = list();   data['stress'] = list()

    for s in S:
        data['greedy_chargers'].append(Results_greedy['Chargers'][s])
        data['two_phase_chargers'].append(Results_two_phase['Chargers'][s])
        data['stress'].append(original_stress[s])
    

    df = pd.DataFrame(data = data)
    df.to_excel(path+ f'/Results/Results.xlsx')


In [6]:
heuristics = ['Greedy assignment', 'Two phase assignment']

Results_greedy = upload_data(heuristics[0])                     # Load data
Results_two_phase = upload_data(heuristics[1])
Results = [Results_two_phase, Results_greedy]

print_general_information(heuristics[0], Results_greedy)        # Print overall results
print_general_information(heuristics[1], Results_two_phase)

export_num_chargers(Results_greedy, Results_two_phase)

### Reconstructing the routes
schedules = {idx:{sc:{} for sc in range(25)} for idx,_ in enumerate(heuristics)}
for idx, heuristic in enumerate(heuristics):
    visited = {sc:[] for sc in range(25)}
    for s in S:
        charger_num = Results[idx][f'Chargers'][s]

        if charger_num == 0:
            continue

        for sc in range(25):
            K, K_s, S_k, a, t = load_pickle(path+"/Data/",sc)

            routes = Results[idx][sc]['routes'][s]

            if len(routes) == 0:
                continue

            for route in routes:
                schedule = {'arrive':[],'wait':[], 'start':[], 'end':[]}

                for pos, vehicle in enumerate(route):
                    visited[sc].append(vehicle)
                    
                    if pos == 0:
                        schedule['arrive'].append(a[vehicle,s])
                        schedule['start'].append(a[vehicle,s])
                        print(t[vehicle,s])
                        schedule['end'].append(t[vehicle,s])
                    else:
                        schedule['arrive'].append(a[vehicle,s])

                        if schedule['end'][-1] > a[vehicle,s]:
                            schedule['wait'].append(schedule['end'][-1] - a[vehicle,s])
                        else:
                            schedule['wait'].append(0)

                        schedule['start'].append(schedule['arrive'][-1] + schedule['wait'][-1])
                        schedule['end'].append(schedule['start'][-1] + t[vehicle,s])


            




---------------- Heuristic Greedy assignment ----------------
- Number of active stations: 165
- Number of chargers: 744 

---------------- Heuristic Two phase assignment ----------------
- Number of active stations: 600
- Number of chargers: 984 

1.0434082948446626
1.5190575560105777
1.413130847019152
1.2063231999246897
1.7255236341619007
1.4790652855599415
0.739703381475133
1.5370091050548456
1.4867915845512656
1.412645475136364
1.5938558925416655
1.7765220431214162
1.214302292360422
0.748327382146095
1.3981070632539698
1.4538094217800828
0.8095982911548467
1.6325884328704277
0.9108662160617376
1.3083627086807335
1.1837541345874143
0.999823400711074
0.7148089866769274
1.3534026131285875
1.6053351843520285
1.6312726032947054
1.33253092458443
1.5977009777263795
0.7414688626954313
1.245710188807655
0.8517320758516209
0.5780522975108235
1.2861044491746663
1.094498960434653
1.1145788080668686
1.527263664811718
1.1509359942158708
1.3112398484864927
1.0622051018638992


KeyError: (3229, 5)

In [21]:
x = [1]
print(x[-1])

1
